# gpstextbook homework 5.4

In [1]:
import pandas as pd
import numpy as np

%pylab inline
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml" # also try "html5" (requires ffmpeg)

Populating the interactive namespace from numpy and matplotlib


In [12]:
frequency_L1 = 1575.42e6  # in Hz
frequency_L2 = 1227.60e6  # in Hz

In [10]:
def read_rinex(path_rinex_file):
    return pd.read_csv(path_rinex_file, engine='python', delimiter='\s+', header=None, 
                       names=['gps_week', 'rx_week', 'prn', 'pseudoL1', 'cyclesL1', 'cyclesL2', 'pseudoP1', 
                      'pseudoP2', 'dopplerL1', 'dopplerL2'])

    

In [17]:
def iono_group_delay(pseudorange_L1, pseudorange_L2, frequency_L1=frequency_L1, frequency_L2=frequency_L2):
    factor = pow(frequency_L2, 2) / (pow(frequency_L1, 2) - pow(frequency_L2, 2))
    
    def get_iono_group_delay(pseudorange_L1, pseudorange_L2):
        return factor * (pseudorange_L2 - pseudorange_L1)
    
    return get_iono_group_delay

In [11]:
path_rinex_file = '/Users/guillaume/Workspace/gnss/data/Data/Pigeon_Point/September_18_2000/Parsed/pp091800.rio'

df = read_rinex(path_rinex_file)

df.head(3)

,gps_week,rx_week,prn,pseudoL1,cyclesL1,cyclesL2,pseudoP1,pseudoP2,dopplerL1,dopplerL2
0,1080.0,86443.0,3.0,2.380454e+07,-8.548223e+06,-6.619810e+06,2.380454e+07,2.380456e+07,2388.252,1860.972
1,1080.0,86443.0,31.0,2.412789e+07,-3.735495e+06,-1.187441e+06,2.412789e+07,2.412791e+07,3899.132,3038.264
2,1080.0,86443.0,20.0,2.352962e+07,-3.212267e+07,-2.446205e+07,2.352962e+07,2.352964e+07,-2686.007,-2092.975


In [49]:
get_iono_group_delay = iono_group_delay(None, None)

df['iono_group_delay'] = df.apply(lambda x: get_iono_group_delay(x['pseudoL1'], x['pseudoP2']), axis=1)

In [50]:
df.head(3)

,gps_week,rx_week,prn,pseudoL1,cyclesL1,cyclesL2,pseudoP1,pseudoP2,dopplerL1,dopplerL2,iono_group_delay
0,1080.0,86443.0,3.0,2.380454e+07,-8.548223e+06,-6.619810e+06,2.380454e+07,2.380456e+07,2388.252,1860.972,23.773293
1,1080.0,86443.0,31.0,2.412789e+07,-3.735495e+06,-1.187441e+06,2.412789e+07,2.412791e+07,3899.132,3038.264,25.912581
2,1080.0,86443.0,20.0,2.352962e+07,-3.212267e+07,-2.446205e+07,2.352962e+07,2.352964e+07,-2686.007,-2092.975,30.412194
